In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

In [16]:
df= pd.read_csv('C:\\Users\\اسما مجدي\\Downloads\\projects\\STC TV\\task 3\\Stc TV Data Set_T3.csv')

In [17]:
df

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,Moana,1,2017-05-21,Animation
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,4,15860,Churchill,2,2017-07-07,Biography
...,...,...,...,...,...,...
1048570,1048570,23716,Extraterrestrial,4,2017-04-22,Horror
1048571,1048571,1776,Friends The One With The Holiday Armadillo,3,2017-11-29,Comedy
1048572,1048572,1918,Masters of Horror Dreams In The Witch House,1,2017-06-15,Horror
1048573,1048573,25737,Two Lovers and a Bear,2,2017-09-11,Drama


In [18]:
# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0','user_id_maped','date_'])

In [19]:
df.columns=['program name','rating','program genre']

In [20]:
df

,program name,rating,program genre
0,100 treets,1,Drama
1,Moana,1,Animation
2,The Mermaid Princess,1,Animation
3,The Mermaid Princess,2,Animation
4,Churchill,2,Biography
...,...,...,...
1048570,Extraterrestrial,4,Horror
1048571,Friends The One With The Holiday Armadillo,3,Comedy
1048572,Masters of Horror Dreams In The Witch House,1,Horror
1048573,Two Lovers and a Bear,2,Drama


In [21]:
df['program name'] = df['program name'].str.split('Ep').str[0].str.strip()

In [22]:
df['program genre'] = df['program genre'].replace('SERIES_NOT_ADDED_UNDER_ANY_GENRE', "Others")
df['program genre'] = df['program genre'].replace('NOT_DEFINED_IN_UMS', "Others")

In [23]:
# Group by program_name and calculate the average rating
df = df.groupby(['program name', 'program genre']).agg({'rating': 'mean'}).reset_index()

In [24]:
df

,program name,program genre,rating
0,,Animation,2.238636
1,#FollowFriday,Thriller,1.784314
2,10 Days in a Madhouse,Thriller,1.707052
3,100 treets,Drama,1.836305
4,101 Dalmatians,Family,1.466102
...,...,...,...
6405,iCarly iWant A World Record,Comedy,3.005814
6406,iCarly iWant More Viewers,Comedy,2.880184
6407,iCarly iWill Date Freddie,Comedy,2.972678
6408,xXx,Action,1.702128


In [25]:
# Encode the program_genre column
label_encoder = LabelEncoder()
df['program_genre_encoded'] = label_encoder.fit_transform(df['program genre'])

# Prepare the features and target
features = df[['rating', 'program_genre_encoded']]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6410 entries, 0 to 6409
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   program name           6410 non-null   object 
 1   program genre          6410 non-null   object 
 2   rating                 6410 non-null   float64
 3   program_genre_encoded  6410 non-null   int32  
dtypes: float64(1), int32(1), object(2)
memory usage: 175.4+ KB


In [27]:
# Fit the KNN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(features)

NearestNeighbors(metric='euclidean')

In [28]:
# Function to find similar movies
def recommendation_system(movie_name, knn, df, label_encoder):
    if movie_name not in df['program name'].values:
        return "Movie not found in the dataset."
    
    movie_index = df[df['program name'] == movie_name].index[0]
    movie_features = features.iloc[movie_index].values.reshape(1, -2)
    
    distances, indices = knn.kneighbors(movie_features)
    
    similar_movies = df.iloc[indices[0]]['program name'].values
    
    # Filter out the input movie from the recommendations
    similar_movies = [movie for movie in similar_movies if movie != movie_name]
    return similar_movies

# Example usage
movie_name = 'American Psycho'
recommended_movies = recommendation_system(movie_name, knn, df, label_encoder)
print(f"Movies similar to {movie_name}: {recommended_movies}")

Movies similar to American Psycho: ['Victor Frankenstein', 'Dirty Dancing', 'The Girl', 'Swiss Army Man']


c:\Users\اسما مجدي\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
